# Рекомендация тарифов

Датасет содержит данные о поведении клиентов, которые уже перешли н тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Необходимо построить модель со значением *accuracy* не менее 0.75 и проверить его на тестовой выборке.

## <u>План проекта</u>

**Обзор данных:**
* Импорт необходимых библиотек;
* Чтение файла и сохранение полученных данных в переменные;
* Получение общей информации (head, info);

**Разделение данных на выборки:**
- тренировочная,
- тестовая,
- валидационная.

**Обучение моделей:**
* DecisionTreeClassifier;
* RandomForestClassifier;
* LogisticRegression.

**Проверка лучшей модели на тестовой выборке:**
* объединение тренировочной и валидационной выборки;
* тестирование модели на ней.

**(бонус) Проверка модели на адекватность**

**Итоги работы**

## Обзор данных

Импортируем библиотеки:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import tree

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

Прочитаем файл:

In [2]:
tariffs = pd.read_csv('/content/users_behavior.csv')
display(tariffs.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Проверим пропуски:

In [3]:
tariffs.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

## Разделение данных на выборки

In [4]:
train_valid, test = train_test_split(tariffs, test_size=0.2, stratify = tariffs['is_ultra'])
train, valid = train_test_split(train_valid, test_size=0.25, stratify = train_valid['is_ultra'])


In [5]:
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']

features_valid = valid.drop(['is_ultra'], axis=1)
target_valid = valid['is_ultra']

features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

ml= [features_train, target_train, features_valid, target_valid, features_test, target_test]
for i in ml:
  print(i.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


## Обучение моделей
на тренировочной и валидационной(чтобы не переобучиться) выборке.

DecisionTreeClassifier:

In [6]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_train)
    result = accuracy_score(target_train, predictions)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

print("Accuracy лучшей модели:", best_result)
print("Depth при лучшем результате:", best_depth)

Accuracy лучшей модели: 0.8895228215767634
Depth при лучшем результате: 10


DecisionTreeClassifier на валидационной выборке:

In [7]:
best_model = None
best_result = 0
best_depth = 0

best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_train = model.predict(features_train)
    result_train = accuracy_score(target_train, predictions_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

print("Accuracy лучшей модели:", best_result)
print("Depth при лучшем результате:", best_depth)


Accuracy лучшей модели: 0.7993779160186625
Depth при лучшем результате: 9


RandomForestClassifier:

In [8]:
best_model = None
best_result = 0
for est in tqdm(range(1, 11)):
    for depth in (range(5,15)):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth, criterion='gini')
        model.fit(features_train, target_train)
        result = model.score(features_train, target_train)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est



print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Depth при лучшем результате:", best_est)

100%|██████████| 10/10 [00:03<00:00,  3.16it/s]

Accuracy наилучшей модели на валидационной выборке: 0.9325726141078838
Depth при лучшем результате: 10


RandomForestClassifier на валидационной выборке:

In [9]:
best_model = None
best_result = 0
for est in tqdm(range(1, 11)):
    for depth in (range(5,15)):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth, criterion='gini')
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est



print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Depth при лучшем результате:", best_est)

100%|██████████| 10/10 [00:05<00:00,  1.87it/s]

Accuracy наилучшей модели на валидационной выборке: 0.8102643856920684
Depth при лучшем результате: 9



Комментарии к RandomForestClassifier:
1. Количество деревьев влияет на правильность обучения на сотую долю%  
2. Чем больше количетво `est`(узлов) тем меньше деревьев будет считаться лучшим результатом.  
    



In [10]:
best_model = None
best_result = 0
for log in range(1, 10):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
    model.fit(features_train, target_train)
    predictions = model.predict(features_train)
    result = model.score(features_train, target_train)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

print("Accuracy лучшей модели:", best_result)



Accuracy лучшей модели: 0.7546680497925311


In [11]:
best_model = None
best_result = 0
for log in range(1, 10):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
    model.fit(features_train, target_train)
    predictions = model.predict(features_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result)


Accuracy наилучшей модели на валидационной выборке: 0.749611197511664


  
Количество итераций мало влияет на правильность. Будь то 200 или 1000.    




Общие выводы по обучению моделей:
    
1. Самый высокий показатель у модели "случайный лес".
2. Смена гиперпараметров моделей меняет показатель правильности на сотые доли%, то есть мало влиеяет на итоговые данные.
3. Модель регрессии отбрасываем - показатели меньше 0,75



## Проверка лучшей модели на тестовой выборке

In [12]:
features = pd.concat([features_train, features_valid])
target = pd.concat([target_train, target_valid])

model = RandomForestClassifier(random_state=12345, n_estimators=9, criterion='gini')
model.fit(features, target)

predictions_test = model.predict(features_test)

test_acc = accuracy_score(target_test, predictions_test)

print("Accuracy test:", test_acc)

Accuracy test: 0.7620528771384136



Тестовая модель больше 0,75.



## (бонус) Проверка модели на адекватность

In [13]:
model = DummyClassifier(strategy='most_frequent', random_state=1)
model.fit(features_train, target_train)
model.predict(features_test)
model.score(features_test, target_test)

0.6936236391912908

## Итоги работы


1. Перед обучением модели поделили выборку на 3 части: обучающая, тестовая, валидационная.
2. Обучили модели DT, RF, LG. Проверила данные на обучающей и валидационной частях. RF оказалась лучшей.
3. Объединили валидационную и обучающую выборку. Проверили модель на тестовой выборке. Результат более 0,78.
4. Минимальный порог верных предсказаний 0,69 - это меньше результатов лучшей модели.